In [2]:
import numpy as np
import pandas as pd
import csv
import torch


In [3]:
# ?preprocessing
def load_clinical(patients):
    threshold = 365
    binary = {}
    death = {}
    times = {}
    clinical_df = patients
    itr = 0
    for pid in patients.index:
        itr += 1
        # assert pid not in clinical_df.index, f"Invalid Patient ID <{pid}>"
        curr_status = clinical_df.loc[pid]['vital_status']
        num_days = 0
        if curr_status == 'Alive':
            num_days = clinical_df.loc[pid]['last_contact_days_to']
            if num_days in ['[Discrepancy]', '[Not Available]']:
                continue
            death[pid] = 0
            times[pid] = num_days
            binary[pid] = 1 * (int(num_days) > threshold)
        elif curr_status == 'Dead':
            num_days = clinical_df.loc[pid]['death_days_to']
            if num_days == '[Not Available]':
                continue
            death[pid] = 1
            times[pid] = num_days
            binary[pid] = 1 * (int(num_days) > threshold)
        else:
            print(pid)

    labels = []
    for idx in death.keys():
        labels.append(tuple((bool(int(death[idx])), int(times[idx]))))
    # dt1 = np.dtype(('bool,float'))
    dt1 = np.dtype(('bool,int'))
    labels = np.array(labels, dtype=dt1)

    # print(list(binary.values()))
    return np.array(list(binary.values())), np.array(death), np.array(times), np.array(labels)


In [4]:
data = pd.read_csv('clinical_with_header.csv')
print(data["vital_status"].value_counts())

x = load_clinical(data)
print(x[0])
print("death or live array :")
deathOrLive = x[1]
print(deathOrLive)
print("time of live per day :")
timeOfLive = x[2]
print(timeOfLive)
print("x[3] :")
print(x[3])


Alive    879
Dead      95
Name: vital_status, dtype: int64
[1 1 1 0 0 0 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 1 1 0 0 1 1 1 0
 0 1 1 0 0 0 1 1 1 0 1 1 1 1 1 0 1 1 0 0 0 1 0 0 1 1 1 1 1 1 1 1 1 1 0 0 0
 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1
 0 0 0 0 0 0 0 0 0 1 1 0 1 1 1 1 1 0 1 1 0 1 1 1 1 1 1 0 1 0 0 0 0 0 0 1 0
 1 0 1 1 1 0 1 1 1 1 0 0 1 0 1 1 0 1 1 1 1 0 1 1 0 1 0 0 0 0 0 0 1 1 0 1 1
 1 0 1 0 1 1 0 1 0 0 1 1 0 1 1 1 0 0 0 0 0 1 1 0 1 1 1 1 1 1 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 0 0 1 1 0 0 1 0 1 1 1 1 0 1 1 1 1 1 1 1 1 1
 1 1 1 1 0 1 0 0 0 1 0 1 1 0 1 1 0 1 1 0 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1
 1 1 1 1 1 0 1 1 1 1 1 0 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 

In [5]:
data.shape

(974, 112)

In [6]:
# data['gender'].value_counts()
data.columns.values

array(['bcr_patient_uuid', 'bcr_patient_barcode', 'form_completion_date',
       'prospective_collection', 'retrospective_collection',
       'birth_days_to', 'gender', 'menopause_status', 'race', 'ethnicity',
       'history_other_malignancy', 'history_neoadjuvant_treatment',
       'tumor_status', 'vital_status', 'last_contact_days_to',
       'death_days_to', 'radiation_treatment_adjuvant',
       'pharmaceutical_tx_adjuvant', 'histologic_diagnosis_other',
       'initial_pathologic_dx_year', 'age_at_diagnosis',
       'method_initial_path_dx', 'method_initial_path_dx_other',
       'surgical_procedure_first', 'first_surgical_procedure_other',
       'margin_status', 'surgery_for_positive_margins',
       'surgery_for_positive_margins_other', 'margin_status_reexcision',
       'axillary_staging_method', 'axillary_staging_method_other',
       'micromet_detection_by_ihc', 'lymph_nodes_examined',
       'lymph_nodes_examined_count', 'lymph_nodes_examined_he_count',
       'lymph_nodes

In [7]:
#! data['bcr_patient_uuid'].value_counts()
#! data['bcr_patient_barcode'].value_counts()
#? data['form_completion_date'].value_counts()
#? data['prospective_collection'].value_counts()
#? data['retrospective_collection'].value_counts()
#? data['birth_days_to'].value_counts()
#* data['gender'].value_counts()
#? data['menopause_status'].value_counts()
#? data['race'].value_counts()
#? data['ethnicity'].value_counts()
#? data['history_other_malignancy'].value_counts()
#? data['history_neoadjuvant_treatment'].value_counts()
#? data['tumor_status'].value_counts()
#* data['vital_status'].value_counts()
#? data['last_contact_days_to'].value_counts()
#! data['death_days_to'].value_counts()
#! data['radiation_treatment_adjuvant'].value_counts()
#! data['pharmaceutical_tx_adjuvant'].value_counts()
#! data['histologic_diagnosis_other'].value_counts()
#? data['initial_pathologic_dx_year'].value_counts()
#* data['age_at_diagnosis'].value_counts()
#? data['method_initial_path_dx'].value_counts()
#! data['method_initial_path_dx_other'].value_counts()
#? data['surgical_procedure_first'].value_counts()
#! data['first_surgical_procedure_other'].value_counts()
#? data['margin_status'].value_counts()
#! data['surgery_for_positive_margins'].value_counts()
#! data['surgery_for_positive_margins_other'].value_counts()
#! data['margin_status_reexcision'].value_counts()
#? data['axillary_staging_method'].value_counts()
#! data['axillary_staging_method_other'].value_counts()
#? data['micromet_detection_by_ihc'].value_counts()
#? data['lymph_nodes_examined'].value_counts()
#? data['lymph_nodes_examined_count'].value_counts()
#? data['lymph_nodes_examined_he_count'].value_counts()
#? data['lymph_nodes_examined_ihc_count'].value_counts()
#? data['ajcc_staging_edition'].value_counts()
#* data['ajcc_tumor_pathologic_pt'].value_counts()
#* data['ajcc_nodes_pathologic_pn'].value_counts()
#* data['ajcc_metastasis_pathologic_pm'].value_counts()
#? data['ajcc_pathologic_tumor_stage'].value_counts()
#! data['metastasis_site'].value_counts()
#! data['metastasis_site_other'].value_counts()
#? data['er_status_by_ihc'].value_counts()
#! data['er_status_ihc_Percent_Positive'].value_counts()
#! data['er_positivity_scale_used'].value_counts()
#! data['er_ihc_score'].value_counts()
#! data['er_positivity_scale_other'].value_counts()
#! data['er_positivity_method'].value_counts()
#? data['pr_status_by_ihc'].value_counts()
#! data['pr_status_ihc_percent_positive'].value_counts()
#! data['pr_positivity_scale_used'].value_counts()
#! data['pr_positivity_ihc_intensity_score'].value_counts()
#! data['pr_positivity_scale_other'].value_counts()
#! data['pr_positivity_define_method'].value_counts()
#! data['her2_status_by_ihc'].value_counts()
#! data['her2_ihc_percent_positive'].value_counts()
#! data['her2_ihc_score'].value_counts()
#! data['her2_positivity_scale_other'].value_counts()
#! data['her2_positivity_method_text'].value_counts()
#! data['her2_fish_status'].value_counts()
#! data['her2_copy_number'].value_counts()
#! data['cent17_copy_number'].value_counts()
#! data['her2_and_cent17_cells_count'].value_counts()
#! data['her2_cent17_ratio'].value_counts()
#! data['her2_and_cent17_scale_other'].value_counts()
#! data['her2_fish_method'].value_counts()
#! data['new_tumor_event_dx_indicator'].value_counts()
#! data['nte_er_status'].value_counts()
#! data['nte_er_status_ihc__positive'].value_counts()
#! data['nte_er_ihc_intensity_score'].value_counts()
#! data['nte_er_positivity_other_scale'].value_counts()
#! data['nte_er_positivity_define_method'].value_counts()
#! data['nte_pr_status_by_ihc'].value_counts()
#! data['nte_pr_status_ihc__positive'].value_counts()
#! data['nte_pr_ihc_intensity_score'].value_counts()
#! data['nte_pr_positivity_other_scale'].value_counts()
#! data['nte_pr_positivity_define_method'].value_counts()
#! data['nte_her2_status'].value_counts()
#! data['nte_her2_status_ihc__positive'].value_counts()
#! data['nte_her2_positivity_ihc_score'].value_counts()
#! data['nte_her2_positivity_other_scale'].value_counts()
#! data['nte_her2_positivity_method'].value_counts()
#! data['nte_her2_fish_status'].value_counts()
#! data['nte_her2_signal_number'].value_counts()
#! data['nte_cent_17_signal_number'].value_counts()
#! data['her2_cent17_counted_cells_count'].value_counts()
#! data['nte_cent_17_her2_ratio'].value_counts()
#! data['nte_cent17_her2_other_scale'].value_counts()
#! data['nte_her2_fish_define_method'].value_counts()
#* data['anatomic_neoplasm_subdivision'].value_counts()
#! data['clinical_M'].value_counts()
#! data['clinical_N'].value_counts()
#! data['clinical_T'].value_counts()
#! data['clinical_stage'].value_counts()
#! data['days_to_initial_pathologic_diagnosis'].value_counts()
#! data['days_to_patient_progression_free'].value_counts()
#! data['days_to_tumor_progression'].value_counts()
#! data['disease_code'].value_counts()
#! data['extranodal_involvement'].value_counts()
#? data['histological_type'].value_counts()
#! data['icd_10'].value_counts()
#* data['icd_o_3_histology'].value_counts()
#! data['icd_o_3_site'].value_counts()
#! data['informed_consent_verified'].value_counts()
#! data['metastatic_tumor_indicator'].value_counts()
#! data['patient_id'].value_counts()
#! data['project_code'].value_counts()
#! data['site_of_primary_tumor_other'].value_counts()
#! data['stage_other'].value_counts()
#* data['tissue_source_site'].value_counts()
#! data['tumor_tissue_site'].value_counts()

In [8]:
#get training and testing data
# training = data[0:900]
# testing = data[900:]

# print training and testing shape
# print(training.shape)
# print(testing.shape)

In [9]:
#? data['form_completion_date'].value_counts()
#? data['prospective_collection'].value_counts()
#? data['retrospective_collection'].value_counts()
#? data['birth_days_to'].value_counts()
#* data['gender'].value_counts()
#? data['menopause_status'].value_counts()
#? data['race'].value_counts()
#? data['ethnicity'].value_counts()
#? data['history_other_malignancy'].value_counts()
#? data['history_neoadjuvant_treatment'].value_counts()
#? data['tumor_status'].value_counts()
#* data['vital_status'].value_counts()
#? data['last_contact_days_to'].value_counts()
#? data['initial_pathologic_dx_year'].value_counts()
#* data['age_at_diagnosis'].value_counts()
#? data['method_initial_path_dx'].value_counts()
#? data['surgical_procedure_first'].value_counts()
#? data['margin_status'].value_counts()
#? data['axillary_staging_method'].value_counts()
#? data['micromet_detection_by_ihc'].value_counts()
#? data['lymph_nodes_examined'].value_counts()
#? data['lymph_nodes_examined_count'].value_counts()
#? data['lymph_nodes_examined_he_count'].value_counts()
#? data['lymph_nodes_examined_ihc_count'].value_counts()
#? data['ajcc_staging_edition'].value_counts()
#* data['ajcc_tumor_pathologic_pt'].value_counts()
#* data['ajcc_nodes_pathologic_pn'].value_counts()
#* data['ajcc_metastasis_pathologic_pm'].value_counts()
#? data['ajcc_pathologic_tumor_stage'].value_counts()
#? data['er_status_by_ihc'].value_counts()
#? data['pr_status_by_ihc'].value_counts()
#* data['anatomic_neoplasm_subdivision'].value_counts()
#? data['histological_type'].value_counts()
#* data['icd_o_3_histology'].value_counts()
#* data['tissue_source_site'].value_counts()

data.drop('er_status_ihc_Percent_Positive', inplace=True, axis=1)
data.drop('pr_status_ihc_percent_positive', inplace=True, axis=1)
data.drop('her2_status_by_ihc', inplace=True, axis=1)
data.drop('her2_ihc_score', inplace=True, axis=1)
data.drop('her2_fish_status', inplace=True, axis=1)
data.drop('metastatic_tumor_indicator', inplace=True, axis=1)
data.drop('bcr_patient_uuid', inplace=True, axis=1)
data.drop('bcr_patient_barcode', inplace=True, axis=1)
data.drop('radiation_treatment_adjuvant', inplace=True, axis=1)
data.drop('death_days_to', inplace=True, axis=1)
data.drop('pharmaceutical_tx_adjuvant', inplace=True, axis=1)
data.drop('histologic_diagnosis_other', inplace=True, axis=1)
data.drop('method_initial_path_dx_other', inplace=True, axis=1)
data.drop('first_surgical_procedure_other', inplace=True, axis=1)
data.drop('surgery_for_positive_margins', inplace=True, axis=1)
data.drop('surgery_for_positive_margins_other', inplace=True, axis=1)
data.drop('margin_status_reexcision', inplace=True, axis=1)
data.drop('axillary_staging_method_other', inplace=True, axis=1)
data.drop('metastasis_site', inplace=True, axis=1)
data.drop('metastasis_site_other', inplace=True, axis=1)
data.drop('er_positivity_scale_used', inplace=True, axis=1)
data.drop('er_ihc_score', inplace=True, axis=1)
data.drop('er_positivity_scale_other', inplace=True, axis=1)
data.drop('er_positivity_method', inplace=True, axis=1)
data.drop('pr_positivity_scale_used', inplace=True, axis=1)
data.drop('pr_positivity_ihc_intensity_score', inplace=True, axis=1)
data.drop('pr_positivity_scale_other', inplace=True, axis=1)
data.drop('pr_positivity_define_method', inplace=True, axis=1)
data.drop('her2_ihc_percent_positive', inplace=True, axis=1)
data.drop('her2_positivity_scale_other', inplace=True, axis=1)
data.drop('her2_positivity_method_text', inplace=True, axis=1)
data.drop('her2_copy_number', inplace=True, axis=1)
data.drop('cent17_copy_number', inplace=True, axis=1)
data.drop('her2_and_cent17_cells_count', inplace=True, axis=1)
data.drop('her2_cent17_ratio', inplace=True, axis=1)
data.drop('her2_and_cent17_scale_other', inplace=True, axis=1)
data.drop('her2_fish_method', inplace=True, axis=1)
data.drop('new_tumor_event_dx_indicator', inplace=True, axis=1)
data.drop('nte_er_status', inplace=True, axis=1)
data.drop('nte_er_status_ihc__positive', inplace=True, axis=1)
data.drop('nte_er_ihc_intensity_score', inplace=True, axis=1)
data.drop('nte_er_positivity_other_scale', inplace=True, axis=1)
data.drop('nte_er_positivity_define_method', inplace=True, axis=1)
data.drop('nte_pr_status_by_ihc', inplace=True, axis=1)
data.drop('nte_pr_status_ihc__positive', inplace=True, axis=1)
data.drop('nte_pr_ihc_intensity_score', inplace=True, axis=1)
data.drop('nte_pr_positivity_other_scale', inplace=True, axis=1)
data.drop('nte_pr_positivity_define_method', inplace=True, axis=1)
data.drop('nte_her2_status', inplace=True, axis=1)
data.drop('nte_her2_status_ihc__positive', inplace=True, axis=1)
data.drop('nte_her2_positivity_ihc_score', inplace=True, axis=1)
data.drop('nte_her2_positivity_other_scale', inplace=True, axis=1)
data.drop('nte_her2_positivity_method', inplace=True, axis=1)
data.drop('nte_her2_fish_status', inplace=True, axis=1)
data.drop('nte_her2_signal_number', inplace=True, axis=1)
data.drop('nte_cent_17_signal_number', inplace=True, axis=1)
data.drop('her2_cent17_counted_cells_count', inplace=True, axis=1)
data.drop('nte_cent_17_her2_ratio', inplace=True, axis=1)
data.drop('nte_cent17_her2_other_scale', inplace=True, axis=1)
data.drop('nte_her2_fish_define_method', inplace=True, axis=1)
data.drop('clinical_M', inplace=True, axis=1)
data.drop('clinical_N', inplace=True, axis=1)
data.drop('clinical_T', inplace=True, axis=1)
data.drop('clinical_stage', inplace=True, axis=1)
data.drop('days_to_initial_pathologic_diagnosis', inplace=True, axis=1)
data.drop('days_to_patient_progression_free', inplace=True, axis=1)
data.drop('days_to_tumor_progression', inplace=True, axis=1)
data.drop('disease_code', inplace=True, axis=1)
data.drop('extranodal_involvement', inplace=True, axis=1)
data.drop('icd_10', inplace=True, axis=1)
data.drop('icd_o_3_site', inplace=True, axis=1)
data.drop('informed_consent_verified', inplace=True, axis=1)
data.drop('patient_id', inplace=True, axis=1)
data.drop('project_code', inplace=True, axis=1)
data.drop('site_of_primary_tumor_other', inplace=True, axis=1)
data.drop('stage_other', inplace=True, axis=1)
data.drop('tumor_tissue_site', inplace=True, axis=1)

In [10]:
data.shape

(974, 35)

In [11]:
#? data['form_completion_date'].value_counts()

years = ['2000', '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013',
         '2014', '2015', '2016', '2017', '2018', '2019', '2020']

for iA, val1 in enumerate(data['form_completion_date'].to_numpy()):
    for iB, val2 in enumerate(years):
        if val2 in val1:
            data['form_completion_date'].to_numpy()[iA] = int(years[iB])

In [12]:
data['form_completion_date'].to_numpy()
data['form_completion_date'].value_counts()

2011    578
2010    183
2013     89
2012     77
2014     40
2015      7
Name: form_completion_date, dtype: int64

In [13]:
# data[data['form_completion_date'] == '2011-5-27'] = '2011'

In [14]:
#? data['prospective_collection'].value_counts()

for index, val1 in enumerate(data['prospective_collection'].to_numpy()):
    if data['prospective_collection'].to_numpy()[index] == 'NO':
        data['prospective_collection'].to_numpy()[index] = 0
    elif data['prospective_collection'].to_numpy()[index] == 'YES':
        data['prospective_collection'].to_numpy()[index] = 1
    else:
        data['prospective_collection'].to_numpy()[index] = 0

data['prospective_collection'].value_counts()

0    641
1    333
Name: prospective_collection, dtype: int64

In [15]:
#? data['retrospective_collection'].value_counts()

for index, val1 in enumerate(data['retrospective_collection'].to_numpy()):
    if data['retrospective_collection'].to_numpy()[index] == 'NO':
        data['retrospective_collection'].to_numpy()[index] = 0
    elif data['retrospective_collection'].to_numpy()[index] == 'YES':
        data['retrospective_collection'].to_numpy()[index] = 1
    else:
        data['retrospective_collection'].to_numpy()[index] = 0

data['retrospective_collection'].value_counts()

1    637
0    337
Name: retrospective_collection, dtype: int64

In [16]:
#? data['birth_days_to'].value_counts()

for index, val1 in enumerate(data['birth_days_to'].to_numpy()):
    if data['birth_days_to'].to_numpy()[index] != '[Not Available]':
        data['birth_days_to'].to_numpy()[index] = int(data['birth_days_to'].to_numpy()[index]) // 1000
    elif data['birth_days_to'].to_numpy()[index] == '[Not Available]':
        data['birth_days_to'].to_numpy()[index] = -27

data['birth_days_to'].value_counts()




-23    88
-19    74
-20    69
-22    69
-24    65
-21    65
-18    63
-27    56
-25    54
-26    54
-17    50
-16    39
-15    38
-28    35
-29    35
-30    35
-14    19
-33    18
-31    16
-13    14
-32     8
-12     5
-11     3
-10     2
Name: birth_days_to, dtype: int64

In [17]:
#? data['menopause_status'].value_counts()

for index, val1 in enumerate(data['menopause_status'].to_numpy()):
    if data['menopause_status'].to_numpy()[
        index] == 'Post (prior bilateral ovariectomy OR >12 mo since LMP with no prior hysterectomy)':
        data['menopause_status'].to_numpy()[index] = 0
    elif data['menopause_status'].to_numpy()[
        index] == 'Pre (<6 months since LMP AND no prior bilateral ovariectomy AND not on estrogen replacement)':
        data['menopause_status'].to_numpy()[index] = 1
    elif data['menopause_status'].to_numpy()[index] == '[Not Available]':
        data['menopause_status'].to_numpy()[index] = 1
    elif data['menopause_status'].to_numpy()[index] == 'Peri (6-12 months since last menstrual period)':
        data['menopause_status'].to_numpy()[index] = 2
    elif data['menopause_status'].to_numpy()[index] == 'Indeterminate (neither Pre or Postmenopausal)':
        data['menopause_status'].to_numpy()[index] = 3
    elif data['menopause_status'].to_numpy()[index] == '[Unknown]':
        data['menopause_status'].to_numpy()[index] = 2
    elif data['menopause_status'].to_numpy()[index] == '[Not Evaluated]':
        data['menopause_status'].to_numpy()[index] = 3

data['menopause_status'].value_counts()



0    627
1    268
2     44
3     35
Name: menopause_status, dtype: int64

In [18]:
#? data['race'].value_counts()

for index, val1 in enumerate(data['race'].to_numpy()):
    if data['race'].to_numpy()[index] == 'WHITE':
        data['race'].to_numpy()[index] = 0
    elif data['race'].to_numpy()[index] == 'BLACK OR AFRICAN AMERICAN':
        data['race'].to_numpy()[index] = 1
    elif data['race'].to_numpy()[index] == '[Not Available]':
        data['race'].to_numpy()[index] = 1
    elif data['race'].to_numpy()[index] == 'ASIAN':
        data['race'].to_numpy()[index] = 2
    elif data['race'].to_numpy()[index] == '[Not Evaluated]':
        data['race'].to_numpy()[index] = 3
    elif data['race'].to_numpy()[index] == 'AMERICAN INDIAN OR ALASKA NATIVE':
        data['race'].to_numpy()[index] = 3

data['race'].value_counts()

0    670
1    242
2     58
3      4
Name: race, dtype: int64

In [19]:
#? data['ethnicity'].value_counts()
for index, val1 in enumerate(data['ethnicity'].to_numpy()):
    if data['ethnicity'].to_numpy()[index] == 'NOT HISPANIC OR LATINO':
        data['ethnicity'].to_numpy()[index] = 1
    elif data['ethnicity'].to_numpy()[index] == '[Not Available]':
        data['ethnicity'].to_numpy()[index] = 1
    elif data['ethnicity'].to_numpy()[index] == 'HISPANIC OR LATINO':
        data['ethnicity'].to_numpy()[index] = 2
    elif data['ethnicity'].to_numpy()[index] == '[Not Evaluated]':
        data['ethnicity'].to_numpy()[index] = 2
    elif data['ethnicity'].to_numpy()[index] == '[Unknown]':
        data['ethnicity'].to_numpy()[index] = 2

data['ethnicity'].value_counts()

1    935
2     39
Name: ethnicity, dtype: int64

In [20]:
#? data['history_other_malignancy'].value_counts()

for index, val1 in enumerate(data['history_other_malignancy'].to_numpy()):
    if data['history_other_malignancy'].to_numpy()[index] == 'No':
        data['history_other_malignancy'].to_numpy()[index] = 1
    elif data['history_other_malignancy'].to_numpy()[index] == 'Yes':
        data['history_other_malignancy'].to_numpy()[index] = 2
    elif data['history_other_malignancy'].to_numpy()[index] == '[Discrepancy]':
        data['history_other_malignancy'].to_numpy()[index] = 2

data['history_other_malignancy'].value_counts()


1    914
2     60
Name: history_other_malignancy, dtype: int64

In [21]:
#? data['history_neoadjuvant_treatment'].value_counts()
for index, val1 in enumerate(data['history_neoadjuvant_treatment'].to_numpy()):
    if data['history_neoadjuvant_treatment'].to_numpy()[index] == 'No':
        data['history_neoadjuvant_treatment'].to_numpy()[index] = 1
    elif data['history_neoadjuvant_treatment'].to_numpy()[index] == 'Yes':
        data['history_neoadjuvant_treatment'].to_numpy()[index] = 2
    elif data['history_neoadjuvant_treatment'].to_numpy()[index] == '[Discrepancy]':
        data['history_neoadjuvant_treatment'].to_numpy()[index] = 2
    elif data['history_neoadjuvant_treatment'].to_numpy()[index] == '[Not Available]':
        data['history_neoadjuvant_treatment'].to_numpy()[index] = 2

data['history_neoadjuvant_treatment'].value_counts()


1    959
2     15
Name: history_neoadjuvant_treatment, dtype: int64

In [22]:
#? data['tumor_status'].value_counts()
for index, val1 in enumerate(data['tumor_status'].to_numpy()):
    if data['tumor_status'].to_numpy()[index] == 'TUMOR FREE':
        data['tumor_status'].to_numpy()[index] = 1
    elif data['tumor_status'].to_numpy()[index] == '[Not Available]':
        data['tumor_status'].to_numpy()[index] = 1
    elif data['tumor_status'].to_numpy()[index] == 'WITH TUMOR':
        data['tumor_status'].to_numpy()[index] = 2
    elif data['tumor_status'].to_numpy()[index] == '[Unknown]':
        data['tumor_status'].to_numpy()[index] = 2

data['tumor_status'].value_counts()


1    875
2     99
Name: tumor_status, dtype: int64

In [23]:
#? data['last_contact_days_to'].value_counts()
c = 0
sum = 0
for index, val1 in enumerate(data['last_contact_days_to'].to_numpy()):
    if data['last_contact_days_to'].to_numpy()[index] != '[Not Available]':
        sum += int(data['last_contact_days_to'].to_numpy()[index])
        c += 1

for index, val1 in enumerate(data['last_contact_days_to'].to_numpy()):
    if data['last_contact_days_to'].to_numpy()[index] == '[Not Available]':
        data['last_contact_days_to'].to_numpy()[index] = sum / c
    elif data['last_contact_days_to'].to_numpy()[index] != '[Not Available]':
        data['last_contact_days_to'].to_numpy()[index] = int(data['last_contact_days_to'].to_numpy()[index])

data['last_contact_days_to'].value_counts()

704.542662     95
0.000000       55
365.000000      8
10.000000       8
34.000000       7
               ..
1826.000000     1
1319.000000     1
1512.000000     1
449.000000      1
2800.000000     1
Name: last_contact_days_to, Length: 590, dtype: int64

In [24]:
#? data['initial_pathologic_dx_year'].value_counts()

for index, val1 in enumerate(data['initial_pathologic_dx_year'].to_numpy()):
    if data['initial_pathologic_dx_year'].to_numpy()[index] == '[Not Available]':
        data['initial_pathologic_dx_year'].to_numpy()[index] = 1990
    elif data['initial_pathologic_dx_year'].to_numpy()[index] != '[Not Available]':
        data['initial_pathologic_dx_year'].to_numpy()[index] = int(data['last_contact_days_to'].to_numpy()[index])

data['initial_pathologic_dx_year'].value_counts()


704     95
0       55
365      8
10       8
34       7
        ..
1826     1
1319     1
1512     1
449      1
2800     1
Name: initial_pathologic_dx_year, Length: 589, dtype: int64

In [25]:
#? data['surgical_procedure_first'].value_counts()
for index, val1 in enumerate(data['surgical_procedure_first'].to_numpy()):
    if data['surgical_procedure_first'].to_numpy()[index] == 'Modified Radical Mastectomy':
        data['surgical_procedure_first'].to_numpy()[index] = 1
    elif data['surgical_procedure_first'].to_numpy()[index] == 'Other':
        data['surgical_procedure_first'].to_numpy()[index] = 2
    elif data['surgical_procedure_first'].to_numpy()[index] == 'Lumpectomy':
        data['surgical_procedure_first'].to_numpy()[index] = 3
    elif data['surgical_procedure_first'].to_numpy()[index] == '[Unknown]':
        data['surgical_procedure_first'].to_numpy()[index] = 2
    elif data['surgical_procedure_first'].to_numpy()[index] == 'Simple Mastectomy':
        data['surgical_procedure_first'].to_numpy()[index] = 4
    elif data['surgical_procedure_first'].to_numpy()[index] == '[Discrepancy]':
        data['surgical_procedure_first'].to_numpy()[index] = 2
    elif data['surgical_procedure_first'].to_numpy()[index] == '[Not Available]':
        data['surgical_procedure_first'].to_numpy()[index] = 1

data['surgical_procedure_first'].value_counts()


1    326
2    266
3    216
4    166
Name: surgical_procedure_first, dtype: int64

In [26]:
#? data['margin_status'].value_counts()

for index, val1 in enumerate(data['margin_status'].to_numpy()):
    if data['margin_status'].to_numpy()[index] == 'Negative':
        data['margin_status'].to_numpy()[index] = -1
    elif data['margin_status'].to_numpy()[index] == 'Positive':
        data['margin_status'].to_numpy()[index] = +1
    elif data['margin_status'].to_numpy()[index] == '[Unknown]':
        data['margin_status'].to_numpy()[index] = +1
    elif data['margin_status'].to_numpy()[index] == 'Close':
        data['margin_status'].to_numpy()[index] = 0
    elif data['margin_status'].to_numpy()[index] == '[Not Available]':
        data['margin_status'].to_numpy()[index] = -1

data['margin_status'].value_counts()

-1    877
 1     70
 0     27
Name: margin_status, dtype: int64

In [27]:
#? data['axillary_staging_method'].value_counts()

for index, val1 in enumerate(data['axillary_staging_method'].to_numpy()):
    if data['axillary_staging_method'].to_numpy()[index] == 'Axillary lymph node dissection alone':
        data['axillary_staging_method'].to_numpy()[index] = 1
    elif data['axillary_staging_method'].to_numpy()[index] == 'Sentinel node biopsy alone':
        data['axillary_staging_method'].to_numpy()[index] = 2
    elif data['axillary_staging_method'].to_numpy()[index] == 'Sentinel lymph node biopsy plus axillary dissection':
        data['axillary_staging_method'].to_numpy()[index] = 3
    elif data['axillary_staging_method'].to_numpy()[index] == '[Not Available]':
        data['axillary_staging_method'].to_numpy()[index] = 1
    elif data['axillary_staging_method'].to_numpy()[index] == 'No axillary staging':
        data['axillary_staging_method'].to_numpy()[index] = 4
    elif data['axillary_staging_method'].to_numpy()[index] == 'Other (specify)':
        data['axillary_staging_method'].to_numpy()[index] = 5
    elif data['axillary_staging_method'].to_numpy()[index] == '[Unknown]':
        data['axillary_staging_method'].to_numpy()[index] = 1

data['axillary_staging_method'].value_counts()


1    489
2    233
3    206
4     29
5     17
Name: axillary_staging_method, dtype: int64

In [28]:
#? data['micromet_detection_by_ihc'].value_counts()

for index, val1 in enumerate(data['micromet_detection_by_ihc'].to_numpy()):
    if data['micromet_detection_by_ihc'].to_numpy()[index] == 'NO':
        data['micromet_detection_by_ihc'].to_numpy()[index] = 0
    elif data['micromet_detection_by_ihc'].to_numpy()[index] == 'YES':
        data['micromet_detection_by_ihc'].to_numpy()[index] = 1
    else:
        data['micromet_detection_by_ihc'].to_numpy()[index] = 0

data['micromet_detection_by_ihc'].value_counts()


0    740
1    234
Name: micromet_detection_by_ihc, dtype: int64

In [29]:
#? data['lymph_nodes_examined'].value_counts()

for index, val1 in enumerate(data['lymph_nodes_examined'].to_numpy()):
    if data['lymph_nodes_examined'].to_numpy()[index] == 'NO':
        data['lymph_nodes_examined'].to_numpy()[index] = 0
    elif data['lymph_nodes_examined'].to_numpy()[index] == 'YES':
        data['lymph_nodes_examined'].to_numpy()[index] = 1
    else:
        data['lymph_nodes_examined'].to_numpy()[index] = 1

data['lymph_nodes_examined'].value_counts()


1    947
0     27
Name: lymph_nodes_examined, dtype: int64

In [30]:
#? data['lymph_nodes_examined_count'].value_counts()

c = 0
sum = 0
for index, val1 in enumerate(data['lymph_nodes_examined_count'].to_numpy()):
    if data['lymph_nodes_examined_count'].to_numpy()[index] != '[Not Available]':
        sum += int(data['lymph_nodes_examined_count'].to_numpy()[index])
        c += 1

for index, val1 in enumerate(data['lymph_nodes_examined_count'].to_numpy()):
    if data['lymph_nodes_examined_count'].to_numpy()[index] == '[Not Available]':
        data['lymph_nodes_examined_count'].to_numpy()[index] = sum / c
    elif data['lymph_nodes_examined_count'].to_numpy()[index] != '[Not Available]':
        data['lymph_nodes_examined_count'].to_numpy()[index] = int(data['lymph_nodes_examined_count'].to_numpy()[index])

data['lymph_nodes_examined_count'].value_counts()


10.501758    121
2.000000     104
1.000000      81
3.000000      62
10.000000     62
4.000000      60
14.000000     35
5.000000      32
15.000000     30
8.000000      30
7.000000      28
11.000000     24
12.000000     23
19.000000     22
13.000000     22
17.000000     22
9.000000      21
6.000000      21
16.000000     21
18.000000     19
23.000000     15
21.000000     14
25.000000     13
22.000000     12
28.000000     10
24.000000     10
26.000000     10
27.000000      8
20.000000      7
30.000000      4
39.000000      4
32.000000      4
31.000000      4
35.000000      3
0.000000       3
29.000000      3
40.000000      2
44.000000      2
36.000000      2
37.000000      1
43.000000      1
38.000000      1
34.000000      1
Name: lymph_nodes_examined_count, dtype: int64

In [31]:
#? data['lymph_nodes_examined_he_count'].value_counts()

c = 0
sum = 0
for index, val1 in enumerate(data['lymph_nodes_examined_he_count'].to_numpy()):
    if data['lymph_nodes_examined_he_count'].to_numpy()[index] != '[Not Available]':
        sum += int(data['lymph_nodes_examined_he_count'].to_numpy()[index])
        c += 1

for index, val1 in enumerate(data['lymph_nodes_examined_he_count'].to_numpy()):
    if data['lymph_nodes_examined_he_count'].to_numpy()[index] == '[Not Available]':
        data['lymph_nodes_examined_he_count'].to_numpy()[index] = sum / c
    elif data['lymph_nodes_examined_he_count'].to_numpy()[index] != '[Not Available]':
        data['lymph_nodes_examined_he_count'].to_numpy()[index] = int(
            data['lymph_nodes_examined_he_count'].to_numpy()[index])

data['lymph_nodes_examined_he_count'].value_counts()


0.00000     403
2.38848     158
1.00000     135
2.00000      74
3.00000      48
4.00000      28
5.00000      26
6.00000      17
8.00000      12
9.00000      10
10.00000     10
12.00000      9
7.00000       5
11.00000      5
14.00000      5
19.00000      4
13.00000      3
18.00000      3
25.00000      3
29.00000      2
22.00000      2
15.00000      2
21.00000      2
27.00000      2
26.00000      1
16.00000      1
24.00000      1
35.00000      1
23.00000      1
28.00000      1
Name: lymph_nodes_examined_he_count, dtype: int64

In [32]:
#? data['lymph_nodes_examined_ihc_count'].value_counts()

c = 0
sum = 0
for index, val1 in enumerate(data['lymph_nodes_examined_ihc_count'].to_numpy()):
    if data['lymph_nodes_examined_ihc_count'].to_numpy()[index] != '[Not Available]':
        sum += int(data['lymph_nodes_examined_ihc_count'].to_numpy()[index])
        c += 1

for index, val1 in enumerate(data['lymph_nodes_examined_ihc_count'].to_numpy()):
    if data['lymph_nodes_examined_ihc_count'].to_numpy()[index] == '[Not Available]':
        data['lymph_nodes_examined_ihc_count'].to_numpy()[index] = sum / c
    elif data['lymph_nodes_examined_ihc_count'].to_numpy()[index] != '[Not Available]':
        data['lymph_nodes_examined_ihc_count'].to_numpy()[index] = int(
            data['lymph_nodes_examined_ihc_count'].to_numpy()[index])

data['lymph_nodes_examined_ihc_count'].value_counts()


0.259868    670
0.000000    250
1.000000     41
2.000000      8
3.000000      2
4.000000      2
8.000000      1
Name: lymph_nodes_examined_ihc_count, dtype: int64

In [33]:
#? data['ajcc_staging_edition'].value_counts()

c = 0
sum = 0
for index, val1 in enumerate(data['ajcc_staging_edition'].to_numpy()):
    if data['ajcc_staging_edition'].to_numpy()[index] != '[Not Available]':
        sum += int(data['ajcc_staging_edition'].to_numpy()[index][0])
        c += 1

for index, val1 in enumerate(data['ajcc_staging_edition'].to_numpy()):
    if data['ajcc_staging_edition'].to_numpy()[index] == '[Not Available]':
        data['ajcc_staging_edition'].to_numpy()[index] = sum / c
    elif data['ajcc_staging_edition'].to_numpy()[index] != '[Not Available]':
        data['ajcc_staging_edition'].to_numpy()[index] = int(data['ajcc_staging_edition'].to_numpy()[index][0])

data['ajcc_staging_edition'].value_counts()

6.000000    404
7.000000    365
6.282864    122
5.000000     49
4.000000     27
3.000000      7
Name: ajcc_staging_edition, dtype: int64

In [34]:
#? data['ajcc_pathologic_tumor_stage'].value_counts()

for index, val1 in enumerate(data['ajcc_pathologic_tumor_stage'].to_numpy()):
    if data['ajcc_pathologic_tumor_stage'].to_numpy()[index] == 'Stage IIA':
        data['ajcc_pathologic_tumor_stage'].to_numpy()[index] = 1
    elif data['ajcc_pathologic_tumor_stage'].to_numpy()[index] == 'Stage IIB':
        data['ajcc_pathologic_tumor_stage'].to_numpy()[index] = 2
    elif data['ajcc_pathologic_tumor_stage'].to_numpy()[index] == 'Stage IIIA':
        data['ajcc_pathologic_tumor_stage'].to_numpy()[index] = 3
    elif data['ajcc_pathologic_tumor_stage'].to_numpy()[index] == 'Stage I':
        data['ajcc_pathologic_tumor_stage'].to_numpy()[index] = 4
    elif data['ajcc_pathologic_tumor_stage'].to_numpy()[index] == 'Stage IA':
        data['ajcc_pathologic_tumor_stage'].to_numpy()[index] = 5
    elif data['ajcc_pathologic_tumor_stage'].to_numpy()[index] == 'Stage IIIC':
        data['ajcc_pathologic_tumor_stage'].to_numpy()[index] = 6
    elif data['ajcc_pathologic_tumor_stage'].to_numpy()[index] == 'Stage IIIB':
        data['ajcc_pathologic_tumor_stage'].to_numpy()[index] = 7
    elif data['ajcc_pathologic_tumor_stage'].to_numpy()[index] == 'Stage IV':
        data['ajcc_pathologic_tumor_stage'].to_numpy()[index] = 8
    elif data['ajcc_pathologic_tumor_stage'].to_numpy()[index] == 'Stage X':
        data['ajcc_pathologic_tumor_stage'].to_numpy()[index] = 9
    elif data['ajcc_pathologic_tumor_stage'].to_numpy()[index] == 'Stage II':
        data['ajcc_pathologic_tumor_stage'].to_numpy()[index] = 10
    elif data['ajcc_pathologic_tumor_stage'].to_numpy()[index] == 'Stage IB':
        data['ajcc_pathologic_tumor_stage'].to_numpy()[index] = 11
    elif data['ajcc_pathologic_tumor_stage'].to_numpy()[index] == 'Stage III':
        data['ajcc_pathologic_tumor_stage'].to_numpy()[index] = 12
    elif data['ajcc_pathologic_tumor_stage'].to_numpy()[index] == '[Discrepancy]':
        data['ajcc_pathologic_tumor_stage'].to_numpy()[index] = 4
    elif data['ajcc_pathologic_tumor_stage'].to_numpy()[index] == '[Not Available]':
        data['ajcc_pathologic_tumor_stage'].to_numpy()[index] = 5

data['ajcc_pathologic_tumor_stage'].value_counts()


1     320
2     229
3     136
4      86
5      80
6      57
7      26
8      17
9      10
10      6
11      5
12      2
Name: ajcc_pathologic_tumor_stage, dtype: int64

In [35]:
#? data['er_status_by_ihc'].value_counts()

for index, val1 in enumerate(data['er_status_by_ihc'].to_numpy()):
    if data['er_status_by_ihc'].to_numpy()[index] == 'Positive':
        data['er_status_by_ihc'].to_numpy()[index] = 1
    elif data['er_status_by_ihc'].to_numpy()[index] == 'Negative':
        data['er_status_by_ihc'].to_numpy()[index] = -1
    elif data['er_status_by_ihc'].to_numpy()[index] == '[Not Evaluated]':
        data['er_status_by_ihc'].to_numpy()[index] = 1
    elif data['er_status_by_ihc'].to_numpy()[index] == 'Indeterminate':
        data['er_status_by_ihc'].to_numpy()[index] = 0

data['er_status_by_ihc'].value_counts()


 1    765
-1    207
 0      2
Name: er_status_by_ihc, dtype: int64

In [36]:
#? data['pr_status_by_ihc'].value_counts()

for index, val1 in enumerate(data['pr_status_by_ihc'].to_numpy()):
    if data['pr_status_by_ihc'].to_numpy()[index] == 'Positive':
        data['pr_status_by_ihc'].to_numpy()[index] = 1
    elif data['pr_status_by_ihc'].to_numpy()[index] == 'Negative':
        data['pr_status_by_ihc'].to_numpy()[index] = -1
    elif data['pr_status_by_ihc'].to_numpy()[index] == '[Not Evaluated]':
        data['pr_status_by_ihc'].to_numpy()[index] = 1
    elif data['pr_status_by_ihc'].to_numpy()[index] == 'Indeterminate':
        data['pr_status_by_ihc'].to_numpy()[index] = 0

data['pr_status_by_ihc'].value_counts()


 1    670
-1    300
 0      4
Name: pr_status_by_ihc, dtype: int64

In [37]:
#? data['histological_type'].value_counts()

for index, val1 in enumerate(data['histological_type'].to_numpy()):
    if data['histological_type'].to_numpy()[index] == 'Infiltrating Ductal Carcinoma':
        data['histological_type'].to_numpy()[index] = 1
    elif data['histological_type'].to_numpy()[index] == 'Infiltrating Lobular Carcinoma':
        data['histological_type'].to_numpy()[index] = 2
    elif data['histological_type'].to_numpy()[index] == 'Other, specify':
        data['histological_type'].to_numpy()[index] = 3
    elif data['histological_type'].to_numpy()[index] == 'Mixed Histology (please specify)':
        data['histological_type'].to_numpy()[index] = 4
    elif data['histological_type'].to_numpy()[index] == 'Mucinous Carcinoma':
        data['histological_type'].to_numpy()[index] = 5
    elif data['histological_type'].to_numpy()[index] == 'Metaplastic Carcinoma':
        data['histological_type'].to_numpy()[index] = 6
    elif data['histological_type'].to_numpy()[index] == 'Medullary Carcinoma':
        data['histological_type'].to_numpy()[index] = 7
    elif data['histological_type'].to_numpy()[index] == 'Infiltrating Carcinoma NOS':
        data['histological_type'].to_numpy()[index] = 8
    elif data['histological_type'].to_numpy()[index] == '[Not Available]':
        data['histological_type'].to_numpy()[index] = 8

data['histological_type'].value_counts()


1    693
2    186
3     40
4     22
5     17
6      8
7      6
8      2
Name: histological_type, dtype: int64

In [38]:
#? data['method_initial_path_dx'].value_counts()

for index, val1 in enumerate(data['method_initial_path_dx'].to_numpy()):
    if data['method_initial_path_dx'].to_numpy()[index] == 'Core needle biopsy':
        data['method_initial_path_dx'].to_numpy()[index] = 1
    elif data['method_initial_path_dx'].to_numpy()[index] == 'Tumor resection':
        data['method_initial_path_dx'].to_numpy()[index] = 2
    elif data['method_initial_path_dx'].to_numpy()[index] == 'Fine needle aspiration biopsy':
        data['method_initial_path_dx'].to_numpy()[index] = 3
    elif data['method_initial_path_dx'].to_numpy()[index] == '[Not Available]':
        data['method_initial_path_dx'].to_numpy()[index] = 1
    elif data['method_initial_path_dx'].to_numpy()[index] == 'Other method, specify:':
        data['method_initial_path_dx'].to_numpy()[index] = 4
    elif data['method_initial_path_dx'].to_numpy()[index] == 'Excisional Biopsy':
        data['method_initial_path_dx'].to_numpy()[index] = 5
    elif data['method_initial_path_dx'].to_numpy()[index] == 'Cytology (e.g. Peritoneal or pleural fluid)':
        data['method_initial_path_dx'].to_numpy()[index] = 6
    elif data['method_initial_path_dx'].to_numpy()[index] == 'Incisional Biopsy':
        data['method_initial_path_dx'].to_numpy()[index] = 7
    elif data['method_initial_path_dx'].to_numpy()[index] == '[Discrepancy]':
        data['method_initial_path_dx'].to_numpy()[index] = 7

data['method_initial_path_dx'].value_counts()


1    625
2    133
3     91
4     60
5     27
6     21
7     17
Name: method_initial_path_dx, dtype: int64

In [39]:
#* data['gender'].value_counts()

for index, val1 in enumerate(data['gender'].to_numpy()):
    if data['gender'].to_numpy()[index] == 'FEMALE':
        data['gender'].to_numpy()[index] = 0
    elif data['gender'].to_numpy()[index] == 'MALE':
        data['gender'].to_numpy()[index] = 1

data['gender'].value_counts()

0    962
1     12
Name: gender, dtype: int64

In [40]:
#* data['icd_o_3_histology'].value_counts()

for index, val1 in enumerate(data['icd_o_3_histology'].to_numpy()):
    data['icd_o_3_histology'].to_numpy()[index] = int(data['icd_o_3_histology'].to_numpy()[index].split("/")[0])

data['icd_o_3_histology'].value_counts()

8500    687
8520    184
8522     21
8523     18
8480     16
8575     13
8503      6
8524      6
8510      6
8507      3
8541      3
8022      2
8090      1
8211      1
8201      1
8010      1
8050      1
8401      1
8013      1
8200      1
9020      1
Name: icd_o_3_histology, dtype: int64

In [41]:
#* data['vital_status'].value_counts()

for index, val1 in enumerate(data['vital_status'].to_numpy()):
    if data['vital_status'].to_numpy()[index] == 'Alive':
        data['vital_status'].to_numpy()[index] = 1
    elif data['vital_status'].to_numpy()[index] == 'Dead':
        data['vital_status'].to_numpy()[index] = 0

data['vital_status'].value_counts()

1    879
0     95
Name: vital_status, dtype: int64

In [42]:
#* data['age_at_diagnosis'].value_counts()

for index, val1 in enumerate(data['age_at_diagnosis'].to_numpy()):
    data['age_at_diagnosis'].to_numpy()[index] = int(data['age_at_diagnosis'].to_numpy()[index])

data['age_at_diagnosis'].value_counts()

62    47
50    33
56    32
61    29
63    29
      ..
86     1
28     1
27     1
26     1
33     1
Name: age_at_diagnosis, Length: 65, dtype: int64

In [43]:
#* data['ajcc_tumor_pathologic_pt'].value_counts()

for index, val1 in enumerate(data['ajcc_tumor_pathologic_pt'].to_numpy()):
    if data['ajcc_tumor_pathologic_pt'].to_numpy()[index] == 'T2':
        data['ajcc_tumor_pathologic_pt'].to_numpy()[index] = 1
    elif data['ajcc_tumor_pathologic_pt'].to_numpy()[index] == 'T1c':
        data['ajcc_tumor_pathologic_pt'].to_numpy()[index] = 2
    elif data['ajcc_tumor_pathologic_pt'].to_numpy()[index] == 'T3':
        data['ajcc_tumor_pathologic_pt'].to_numpy()[index] = 3
    elif data['ajcc_tumor_pathologic_pt'].to_numpy()[index] == 'T1':
        data['ajcc_tumor_pathologic_pt'].to_numpy()[index] = 4
    elif data['ajcc_tumor_pathologic_pt'].to_numpy()[index] == 'T4b':
        data['ajcc_tumor_pathologic_pt'].to_numpy()[index] = 5
    elif data['ajcc_tumor_pathologic_pt'].to_numpy()[index] == 'T1b':
        data['ajcc_tumor_pathologic_pt'].to_numpy()[index] = 6
    elif data['ajcc_tumor_pathologic_pt'].to_numpy()[index] == 'T4':
        data['ajcc_tumor_pathologic_pt'].to_numpy()[index] = 7
    elif data['ajcc_tumor_pathologic_pt'].to_numpy()[index] == 'T4d':
        data['ajcc_tumor_pathologic_pt'].to_numpy()[index] = 8
    elif data['ajcc_tumor_pathologic_pt'].to_numpy()[index] == 'TX':
        data['ajcc_tumor_pathologic_pt'].to_numpy()[index] = 9
    elif data['ajcc_tumor_pathologic_pt'].to_numpy()[index] == 'T2b':
        data['ajcc_tumor_pathologic_pt'].to_numpy()[index] = 10
    elif data['ajcc_tumor_pathologic_pt'].to_numpy()[index] == 'T1a':
        data['ajcc_tumor_pathologic_pt'].to_numpy()[index] = 11
    elif data['ajcc_tumor_pathologic_pt'].to_numpy()[index] == 'T3a':
        data['ajcc_tumor_pathologic_pt'].to_numpy()[index] = 12

data['ajcc_tumor_pathologic_pt'].value_counts()

1     559
2     196
3     128
4      39
5      26
6      11
7       9
8       2
9       1
10      1
11      1
12      1
Name: ajcc_tumor_pathologic_pt, dtype: int64

In [44]:
#* data['ajcc_nodes_pathologic_pn'].value_counts()

for index, val1 in enumerate(data['ajcc_nodes_pathologic_pn'].to_numpy()):
    if data['ajcc_nodes_pathologic_pn'].to_numpy()[index] == 'N0':
        data['ajcc_nodes_pathologic_pn'].to_numpy()[index] = 1
    elif data['ajcc_nodes_pathologic_pn'].to_numpy()[index] == 'N1a':
        data['ajcc_nodes_pathologic_pn'].to_numpy()[index] = 2
    elif data['ajcc_nodes_pathologic_pn'].to_numpy()[index] == 'N0 (i-)':
        data['ajcc_nodes_pathologic_pn'].to_numpy()[index] = 3
    elif data['ajcc_nodes_pathologic_pn'].to_numpy()[index] == 'N1':
        data['ajcc_nodes_pathologic_pn'].to_numpy()[index] = 4
    elif data['ajcc_nodes_pathologic_pn'].to_numpy()[index] == 'N2a':
        data['ajcc_nodes_pathologic_pn'].to_numpy()[index] = 5
    elif data['ajcc_nodes_pathologic_pn'].to_numpy()[index] == 'N2':
        data['ajcc_nodes_pathologic_pn'].to_numpy()[index] = 6
    elif data['ajcc_nodes_pathologic_pn'].to_numpy()[index] == 'N3a':
        data['ajcc_nodes_pathologic_pn'].to_numpy()[index] = 7
    elif data['ajcc_nodes_pathologic_pn'].to_numpy()[index] == 'N1mi':
        data['ajcc_nodes_pathologic_pn'].to_numpy()[index] = 8
    elif data['ajcc_nodes_pathologic_pn'].to_numpy()[index] == 'N1b':
        data['ajcc_nodes_pathologic_pn'].to_numpy()[index] = 9
    elif data['ajcc_nodes_pathologic_pn'].to_numpy()[index] == 'N3':
        data['ajcc_nodes_pathologic_pn'].to_numpy()[index] = 10
    elif data['ajcc_nodes_pathologic_pn'].to_numpy()[index] == 'N0 (i+)':
        data['ajcc_nodes_pathologic_pn'].to_numpy()[index] = 11
    elif data['ajcc_nodes_pathologic_pn'].to_numpy()[index] == 'NX':
        data['ajcc_nodes_pathologic_pn'].to_numpy()[index] = 12
    elif data['ajcc_nodes_pathologic_pn'].to_numpy()[index] == 'N1c':
        data['ajcc_nodes_pathologic_pn'].to_numpy()[index] = 13
    elif data['ajcc_nodes_pathologic_pn'].to_numpy()[index] == 'N3c':
        data['ajcc_nodes_pathologic_pn'].to_numpy()[index] = 14
    elif data['ajcc_nodes_pathologic_pn'].to_numpy()[index] == 'N3b':
        data['ajcc_nodes_pathologic_pn'].to_numpy()[index] = 15

data['ajcc_nodes_pathologic_pn'].value_counts()

1     304
2     146
3     132
4     116
5      57
6      46
7      42
8      32
9      31
10     24
11     22
12     18
13      2
14      1
15      1
Name: ajcc_nodes_pathologic_pn, dtype: int64

In [45]:
#* data['ajcc_metastasis_pathologic_pm'].value_counts()
for index, val1 in enumerate(data['ajcc_metastasis_pathologic_pm'].to_numpy()):
    if data['ajcc_metastasis_pathologic_pm'].to_numpy()[index] == 'M0':
        data['ajcc_metastasis_pathologic_pm'].to_numpy()[index] = 1
    elif data['ajcc_metastasis_pathologic_pm'].to_numpy()[index] == 'MX':
        data['ajcc_metastasis_pathologic_pm'].to_numpy()[index] = 2
    elif data['ajcc_metastasis_pathologic_pm'].to_numpy()[index] == 'M1':
        data['ajcc_metastasis_pathologic_pm'].to_numpy()[index] = 3
    elif data['ajcc_metastasis_pathologic_pm'].to_numpy()[index] == 'cM0 (i+)':
        data['ajcc_metastasis_pathologic_pm'].to_numpy()[index] = 4

data['ajcc_metastasis_pathologic_pm'].value_counts()

1    812
2    140
3     19
4      3
Name: ajcc_metastasis_pathologic_pm, dtype: int64

In [46]:
#* data['anatomic_neoplasm_subdivision'].value_counts()

for index, val1 in enumerate(data['anatomic_neoplasm_subdivision'].to_numpy()):
    if data['anatomic_neoplasm_subdivision'].to_numpy()[index] == 'Left Upper Outer Quadrant':
        data['anatomic_neoplasm_subdivision'].to_numpy()[index] = 1
    elif data['anatomic_neoplasm_subdivision'].to_numpy()[index] == 'Left':
        data['anatomic_neoplasm_subdivision'].to_numpy()[index] = 2
    elif data['anatomic_neoplasm_subdivision'].to_numpy()[index] == 'Right Upper Outer Quadrant':
        data['anatomic_neoplasm_subdivision'].to_numpy()[index] = 3
    elif data['anatomic_neoplasm_subdivision'].to_numpy()[index] == 'Right':
        data['anatomic_neoplasm_subdivision'].to_numpy()[index] = 4
    elif data['anatomic_neoplasm_subdivision'].to_numpy()[index] == 'Left Upper Inner Quadrant':
        data['anatomic_neoplasm_subdivision'].to_numpy()[index] = 5
    elif data['anatomic_neoplasm_subdivision'].to_numpy()[index] == 'Right Upper Inner Quadrant':
        data['anatomic_neoplasm_subdivision'].to_numpy()[index] = 6
    elif data['anatomic_neoplasm_subdivision'].to_numpy()[index] == 'Right Lower Outer Quadrant':
        data['anatomic_neoplasm_subdivision'].to_numpy()[index] = 7
    elif data['anatomic_neoplasm_subdivision'].to_numpy()[index] == 'Left Lower Outer Quadrant':
        data['anatomic_neoplasm_subdivision'].to_numpy()[index] = 8
    elif data['anatomic_neoplasm_subdivision'].to_numpy()[index] == 'Left Upper Outer Quadrant|Left':
        data['anatomic_neoplasm_subdivision'].to_numpy()[index] = 9
    elif data['anatomic_neoplasm_subdivision'].to_numpy()[index] == 'Right Upper Outer Quadrant|Right':
        data['anatomic_neoplasm_subdivision'].to_numpy()[index] = 10
    elif data['anatomic_neoplasm_subdivision'].to_numpy()[index] == 'Right Lower Inner Quadrant':
        data['anatomic_neoplasm_subdivision'].to_numpy()[index] = 11
    elif data['anatomic_neoplasm_subdivision'].to_numpy()[index] == 'Left Lower Inner Quadrant':
        data['anatomic_neoplasm_subdivision'].to_numpy()[index] = 12
    elif data['anatomic_neoplasm_subdivision'].to_numpy()[index] == 'Right Upper Inner Quadrant|Right':
        data['anatomic_neoplasm_subdivision'].to_numpy()[index] = 13
    elif data['anatomic_neoplasm_subdivision'].to_numpy()[index] == 'Left|Left Upper Outer Quadrant':
        data['anatomic_neoplasm_subdivision'].to_numpy()[index] = 14
    elif data['anatomic_neoplasm_subdivision'].to_numpy()[
        index] == 'Left Upper Inner Quadrant|Left Upper Outer Quadrant':
        data['anatomic_neoplasm_subdivision'].to_numpy()[index] = 15
    elif data['anatomic_neoplasm_subdivision'].to_numpy()[index] == 'Right|Right Upper Outer Quadrant':
        data['anatomic_neoplasm_subdivision'].to_numpy()[index] = 16
    elif data['anatomic_neoplasm_subdivision'].to_numpy()[index] == 'Right|Right Upper Inner Quadrant':
        data['anatomic_neoplasm_subdivision'].to_numpy()[index] = 17
    elif data['anatomic_neoplasm_subdivision'].to_numpy()[
        index] == 'Left Upper Outer Quadrant|Left Lower Outer Quadrant':
        data['anatomic_neoplasm_subdivision'].to_numpy()[index] = 18
    elif data['anatomic_neoplasm_subdivision'].to_numpy()[
        index] == 'Right Upper Inner Quadrant|Right Upper Outer Quadrant|Right Lower Inner Quadrant|Right Lower Outer Quadrant|Right':
        data['anatomic_neoplasm_subdivision'].to_numpy()[index] = 19
    elif data['anatomic_neoplasm_subdivision'].to_numpy()[
        index] == 'Right Upper Inner Quadrant|Right Upper Outer Quadrant':
        data['anatomic_neoplasm_subdivision'].to_numpy()[index] = 20
    elif data['anatomic_neoplasm_subdivision'].to_numpy()[
        index] == 'Left Upper Outer Quadrant|Left Lower Outer Quadrant|Left':
        data['anatomic_neoplasm_subdivision'].to_numpy()[index] = 21
    elif data['anatomic_neoplasm_subdivision'].to_numpy()[index] == 'Left Lower Outer Quadrant|Left':
        data['anatomic_neoplasm_subdivision'].to_numpy()[index] = 22
    elif data['anatomic_neoplasm_subdivision'].to_numpy()[
        index] == 'Left Upper Inner Quadrant|Left Upper Outer Quadrant|Left':
        data['anatomic_neoplasm_subdivision'].to_numpy()[index] = 23
    elif data['anatomic_neoplasm_subdivision'].to_numpy()[index] == 'Left Lower Inner Quadrant|Left':
        data['anatomic_neoplasm_subdivision'].to_numpy()[index] = 24
    elif data['anatomic_neoplasm_subdivision'].to_numpy()[
        index] == 'Right Upper Outer Quadrant|Right Lower Outer Quadrant':
        data['anatomic_neoplasm_subdivision'].to_numpy()[index] = 25
    elif data['anatomic_neoplasm_subdivision'].to_numpy()[
        index] == 'Left Lower Inner Quadrant|Left Lower Outer Quadrant|Left':
        data['anatomic_neoplasm_subdivision'].to_numpy()[index] = 26
    elif data['anatomic_neoplasm_subdivision'].to_numpy()[
        index] == 'Right Upper Inner Quadrant|Right Lower Inner Quadrant|Right':
        data['anatomic_neoplasm_subdivision'].to_numpy()[index] = 27
    elif data['anatomic_neoplasm_subdivision'].to_numpy()[
        index] == 'Right Upper Inner Quadrant|Right Upper Outer Quadrant|Right':
        data['anatomic_neoplasm_subdivision'].to_numpy()[index] = 28
    elif data['anatomic_neoplasm_subdivision'].to_numpy()[index] == 'Left Upper Inner Quadrant|Left':
        data['anatomic_neoplasm_subdivision'].to_numpy()[index] = 29
    elif data['anatomic_neoplasm_subdivision'].to_numpy()[index] == 'Right Lower Outer Quadrant|Right':
        data['anatomic_neoplasm_subdivision'].to_numpy()[index] = 30
    elif data['anatomic_neoplasm_subdivision'].to_numpy()[
        index] == 'Right Lower Inner Quadrant|Right Lower Outer Quadrant|Right':
        data['anatomic_neoplasm_subdivision'].to_numpy()[index] = 31
    elif data['anatomic_neoplasm_subdivision'].to_numpy()[index] == 'Right|Right Lower Inner Quadrant':
        data['anatomic_neoplasm_subdivision'].to_numpy()[index] = 32
    elif data['anatomic_neoplasm_subdivision'].to_numpy()[index] == 'Right|Right Lower Outer Quadrant':
        data['anatomic_neoplasm_subdivision'].to_numpy()[index] = 33
    elif data['anatomic_neoplasm_subdivision'].to_numpy()[
        index] == 'Right Lower Inner Quadrant|Right Lower Outer Quadrant':
        data['anatomic_neoplasm_subdivision'].to_numpy()[index] = 34
    elif data['anatomic_neoplasm_subdivision'].to_numpy()[
        index] == 'Left Upper Inner Quadrant|Left Lower Inner Quadrant':
        data['anatomic_neoplasm_subdivision'].to_numpy()[index] = 35
    elif data['anatomic_neoplasm_subdivision'].to_numpy()[
        index] == 'Right Upper Inner Quadrant|Right Lower Inner Quadrant':
        data['anatomic_neoplasm_subdivision'].to_numpy()[index] = 36
    elif data['anatomic_neoplasm_subdivision'].to_numpy()[
        index] == 'Left Upper Inner Quadrant|Left Upper Outer Quadrant|Left Lower Inner Quadrant|Left Lower Outer Quadrant':
        data['anatomic_neoplasm_subdivision'].to_numpy()[index] = 37
    elif data['anatomic_neoplasm_subdivision'].to_numpy()[index] == 'Left|Left Upper Inner Quadrant':
        data['anatomic_neoplasm_subdivision'].to_numpy()[index] = 38
    elif data['anatomic_neoplasm_subdivision'].to_numpy()[
        index] == 'Left Upper Inner Quadrant|Left Upper Outer Quadrant|Left Lower Inner Quadrant|Left Lower Outer Quadrant|Left':
        data['anatomic_neoplasm_subdivision'].to_numpy()[index] = 39
    elif data['anatomic_neoplasm_subdivision'].to_numpy()[index] == 'Left|Left Lower Inner Quadrant':
        data['anatomic_neoplasm_subdivision'].to_numpy()[index] = 40
    elif data['anatomic_neoplasm_subdivision'].to_numpy()[
        index] == 'Right Upper Outer Quadrant|Right Lower Outer Quadrant|Right':
        data['anatomic_neoplasm_subdivision'].to_numpy()[index] = 41
    elif data['anatomic_neoplasm_subdivision'].to_numpy()[
        index] == 'Left Lower Inner Quadrant|Left Lower Outer Quadrant':
        data['anatomic_neoplasm_subdivision'].to_numpy()[index] = 42
    elif data['anatomic_neoplasm_subdivision'].to_numpy()[
        index] == 'Left|Left Upper Inner Quadrant|Left Upper Outer Quadrant|Left Lower Inner Quadrant|Left Lower Outer Quadrant':
        data['anatomic_neoplasm_subdivision'].to_numpy()[index] = 43
    elif data['anatomic_neoplasm_subdivision'].to_numpy()[
        index] == 'Left Upper Inner Quadrant|Left Lower Inner Quadrant|Left Lower Outer Quadrant|Left':
        data['anatomic_neoplasm_subdivision'].to_numpy()[index] = 44
    elif data['anatomic_neoplasm_subdivision'].to_numpy()[
        index] == 'Right Upper Outer Quadrant|Right Lower Inner Quadrant|Right Lower Outer Quadrant':
        data['anatomic_neoplasm_subdivision'].to_numpy()[index] = 45
    elif data['anatomic_neoplasm_subdivision'].to_numpy()[index] == 'Right Lower Inner Quadrant|Right':
        data['anatomic_neoplasm_subdivision'].to_numpy()[index] = 46
    elif data['anatomic_neoplasm_subdivision'].to_numpy()[
        index] == 'Left Upper Inner Quadrant|Left Upper Outer Quadrant|Left Lower Outer Quadrant|Left':
        data['anatomic_neoplasm_subdivision'].to_numpy()[index] = 47
    elif data['anatomic_neoplasm_subdivision'].to_numpy()[
        index] == 'Left Upper Inner Quadrant|Left Lower Inner Quadrant|Left':
        data['anatomic_neoplasm_subdivision'].to_numpy()[index] = 48
    elif data['anatomic_neoplasm_subdivision'].to_numpy()[
        index] == 'Left Upper Inner Quadrant|Left Upper Outer Quadrant|Left Lower Inner Quadrant|Left':
        data['anatomic_neoplasm_subdivision'].to_numpy()[index] = 49
    elif data['anatomic_neoplasm_subdivision'].to_numpy()[
        index] == 'Left Upper Outer Quadrant|Left Lower Inner Quadrant|Left Lower Outer Quadrant':
        data['anatomic_neoplasm_subdivision'].to_numpy()[index] = 50

data['anatomic_neoplasm_subdivision'].value_counts()

1     163
2     157
3     140
4     138
5      50
6      46
7      42
8      31
9      25
10     21
11     19
12     17
13     11
14     11
15      9
16      8
17      6
18      5
19      5
20      5
21      4
22      4
23      4
24      4
25      4
26      3
27      3
28      3
29      3
30      3
31      2
32      2
33      2
34      2
35      2
36      2
37      2
38      2
39      2
40      2
41      1
42      1
43      1
44      1
45      1
46      1
47      1
48      1
49      1
50      1
Name: anatomic_neoplasm_subdivision, dtype: int64

In [47]:
#* data['tissue_source_site'].value_counts()

for index, val1 in enumerate(data['tissue_source_site'].to_numpy()):
    if data['tissue_source_site'].to_numpy()[index] == 'BH':
        data['tissue_source_site'].to_numpy()[index] = 1
    elif data['tissue_source_site'].to_numpy()[index] == 'A2':
        data['tissue_source_site'].to_numpy()[index] = 2
    elif data['tissue_source_site'].to_numpy()[index] == 'A8':
        data['tissue_source_site'].to_numpy()[index] = 3
    elif data['tissue_source_site'].to_numpy()[index] == 'E2':
        data['tissue_source_site'].to_numpy()[index] = 4
    elif data['tissue_source_site'].to_numpy()[index] == 'D8':
        data['tissue_source_site'].to_numpy()[index] = 5
    elif data['tissue_source_site'].to_numpy()[index] == 'AR':
        data['tissue_source_site'].to_numpy()[index] = 6
    elif data['tissue_source_site'].to_numpy()[index] == 'E9':
        data['tissue_source_site'].to_numpy()[index] = 7
    elif data['tissue_source_site'].to_numpy()[index] == 'C8':
        data['tissue_source_site'].to_numpy()[index] = 8
    elif data['tissue_source_site'].to_numpy()[index] == 'AC':
        data['tissue_source_site'].to_numpy()[index] = 9
    elif data['tissue_source_site'].to_numpy()[index] == 'B6':
        data['tissue_source_site'].to_numpy()[index] = 10
    elif data['tissue_source_site'].to_numpy()[index] == 'EW':
        data['tissue_source_site'].to_numpy()[index] = 11
    elif data['tissue_source_site'].to_numpy()[index] == 'A7':
        data['tissue_source_site'].to_numpy()[index] = 12
    elif data['tissue_source_site'].to_numpy()[index] == 'AO':
        data['tissue_source_site'].to_numpy()[index] = 13
    elif data['tissue_source_site'].to_numpy()[index] == 'AN':
        data['tissue_source_site'].to_numpy()[index] = 14
    elif data['tissue_source_site'].to_numpy()[index] == 'GM':
        data['tissue_source_site'].to_numpy()[index] = 15
    elif data['tissue_source_site'].to_numpy()[index] == 'LL':
        data['tissue_source_site'].to_numpy()[index] = 16
    elif data['tissue_source_site'].to_numpy()[index] == 'OL':
        data['tissue_source_site'].to_numpy()[index] = 17
    elif data['tissue_source_site'].to_numpy()[index] == 'A1':
        data['tissue_source_site'].to_numpy()[index] = 18
    elif data['tissue_source_site'].to_numpy()[index] == 'AQ':
        data['tissue_source_site'].to_numpy()[index] = 19
    elif data['tissue_source_site'].to_numpy()[index] == 'LD':
        data['tissue_source_site'].to_numpy()[index] = 20
    elif data['tissue_source_site'].to_numpy()[index] == 'PL':
        data['tissue_source_site'].to_numpy()[index] = 21
    elif data['tissue_source_site'].to_numpy()[index] == 'S3':
        data['tissue_source_site'].to_numpy()[index] = 22
    elif data['tissue_source_site'].to_numpy()[index] == 'PE':
        data['tissue_source_site'].to_numpy()[index] = 23
    elif data['tissue_source_site'].to_numpy()[index] == '3C':
        data['tissue_source_site'].to_numpy()[index] = 24
    elif data['tissue_source_site'].to_numpy()[index] == 'JL':
        data['tissue_source_site'].to_numpy()[index] = 25
    elif data['tissue_source_site'].to_numpy()[index] == 'HN':
        data['tissue_source_site'].to_numpy()[index] = 26
    elif data['tissue_source_site'].to_numpy()[index] == 'GI':
        data['tissue_source_site'].to_numpy()[index] = 27
    elif data['tissue_source_site'].to_numpy()[index] == 'WT':
        data['tissue_source_site'].to_numpy()[index] = 28
    elif data['tissue_source_site'].to_numpy()[index] == 'XX':
        data['tissue_source_site'].to_numpy()[index] = 29
    elif data['tissue_source_site'].to_numpy()[index] == 'Z7':
        data['tissue_source_site'].to_numpy()[index] = 30
    elif data['tissue_source_site'].to_numpy()[index] == 'LQ':
        data['tissue_source_site'].to_numpy()[index] = 31
    elif data['tissue_source_site'].to_numpy()[index] == 'MS':
        data['tissue_source_site'].to_numpy()[index] = 32
    elif data['tissue_source_site'].to_numpy()[index] == 'OK':
        data['tissue_source_site'].to_numpy()[index] = 33
    elif data['tissue_source_site'].to_numpy()[index] == '4H':
        data['tissue_source_site'].to_numpy()[index] = 34
    elif data['tissue_source_site'].to_numpy()[index] == '5T':
        data['tissue_source_site'].to_numpy()[index] = 35

data['tissue_source_site'].value_counts()

1     133
2      98
3      82
4      78
5      78
6      69
7      56
8      47
9      46
10     46
11     42
12     41
13     35
14     20
15     19
16     19
17     13
18     12
19      5
20      5
21      4
22      3
23      3
24      3
25      2
26      2
27      2
28      2
29      2
30      2
31      1
32      1
33      1
34      1
35      1
Name: tissue_source_site, dtype: int64

In [48]:
data.shape

(974, 35)

In [49]:
# add a column to dataFrame (binary from preprocessing)
data.insert(35, 'survive_for_more_than_a_year', x[0])

In [50]:
data['survive_for_more_than_a_year'].value_counts()

1    507
0    467
Name: survive_for_more_than_a_year, dtype: int64

In [51]:
from sklearn.model_selection import train_test_split

features = data.copy()
labels = features.pop('survive_for_more_than_a_year')

In [52]:
print(np.array(features).shape)
print(np.array(labels).shape)
np.array(labels)

(974, 35)
(974,)


array([1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0,
       1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1,
       1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0,
       1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1,
       0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0,
       1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0,
       0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [53]:
X_train, X_test, y_train, y_test = train_test_split(np.array(features), np.array(labels), test_size=0.2, random_state=1)

print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

features = np.array(features)
features

(779, 35)
(195, 35)
(779,)
(195,)


array([[2014, 0, 1, ..., 1, 8500, 24],
       [2014, 0, 1, ..., 1, 8500, 24],
       [2014, 0, 1, ..., 1, 8500, 24],
       ...,
       [2014, 0, 1, ..., 2, 8520, 29],
       [2014, 0, 1, ..., 2, 8520, 30],
       [2014, 0, 1, ..., 2, 8022, 30]], dtype=object)

In [54]:
X_train = np.asarray(X_train).astype(np.float32)
y_train = np.asarray(y_train).astype(np.float32)

print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(779, 35)
(195, 35)
(779,)
(195,)


In [55]:
from tensorflow import keras

input_layer = keras.layers.Input(shape=(X_train.shape[1]))
hidden1 = keras.layers.Dense(units=8, activation='relu')(input_layer)
hidden2 = keras.layers.Dense(units=8, activation='relu')(hidden1)
# hidden3 = keras.layers.Dense(units=128, activation='relu')(hidden2)
output = keras.layers.Dense(1)(hidden2)

In [63]:
import tensorflow as tf

model = keras.Sequential([
    keras.layers.Input(shape=(X_train.shape[1])),
    keras.layers.Dense(units=64, activation='relu'),
    keras.layers.Dense(units=64, activation='relu'),
    keras.layers.Dense(units=64, activation='relu'),
    keras.layers.Dense(units=64, activation='relu'),
    keras.layers.Dense(4)
])

model.compile(loss=tf.losses.MeanSquaredError(),
              optimizer=tf.optimizers.Adam())

history = model.fit(X_train, y_train, epochs=100)

Epoch 1/100
25/25 [==============================] - 0s 2ms/step - loss: 14486.8545
Epoch 2/100
25/25 [==============================] - 0s 2ms/step - loss: 251.0284
Epoch 3/100
25/25 [==============================] - 0s 2ms/step - loss: 42.1128
Epoch 4/100
25/25 [==============================] - 0s 2ms/step - loss: 14.4000
Epoch 5/100
25/25 [==============================] - 0s 2ms/step - loss: 8.0194
Epoch 6/100
25/25 [==============================] - 0s 2ms/step - loss: 5.5167
Epoch 7/100
25/25 [==============================] - 0s 2ms/step - loss: 4.7495
Epoch 8/100
25/25 [==============================] - 0s 1ms/step - loss: 3.9893
Epoch 9/100
25/25 [==============================] - 0s 2ms/step - loss: 3.4522
Epoch 10/100
25/25 [==============================] - 0s 2ms/step - loss: 3.1305
Epoch 11/100
25/25 [==============================] - 0s 1ms/step - loss: 2.8098
Epoch 12/100
25/25 [==============================] - 0s 2ms/step - loss: 2.6153
Epoch 13/100
25/25 [=========